## Tarea
Va a consistir de 4 pasos basados en lo que aprendimos de como manipular los INEGI shape files.

## Primero
#### Vas a escoger 5 estados y vas a graficar sus vecinos como vimos en el otro archivo. A cada uno de estos grupos los llamaremos grupos de estados.
#### Usa el shapefile de los estados


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, LineString, Polygon
from shapely.wkt import loads
import fiona
import matplotlib.pyplot as plt

In [ ]:
#Pon el path a tu shapefile de la carpeta de Estados que descargaste. Ejemplo:
file_pathAShapefile_Estados = 'ShapeFilesFiles/Estados/conjunto_de_datos/00ent.shp'
#Pon el path a tu shapefile de la carpeta de servicios_de_salud_shp que descargaste. Ejemplo:
file_pathAShapefile_ServicioSalud = 'ShapeFilesFiles/conjunto_de_datos/denue_inegi_62_.shp'
#Leer shapefiles que usaremos
gdf_mexico = gpd.read_file(file_pathAShapefile_Estados) #Leer shapefile de estados
gdf_salud = gpd.read_file(file_pathAShapefile_ServicioSalud) #Leer shapefile de servicios de salud


In [ ]:
vecinos_dict = {} #Diccionario para almacenar los vecinos de cada estado
# Iterar sobre cada estado en el dataframe
for index, row in gdf_mexico.iterrows(): 
    # Obtener la geometría del estado actual
    estado_geom = row['geometry']
    
    # Encontrar los estados que intersectan con el estado actual (incluye fronteras y superposiciones)
    gdf_vecinos = gdf_mexico[gdf_mexico.geometry.intersects(estado_geom)] 
    
    # Eliminar el estado mismo de la lista de vecinos (si se incluye)
    gdf_vecinos = gdf_vecinos[gdf_vecinos['NOMGEO'] != row['NOMGEO']] 
    
    # Obtener los nombres de los vecinos
    vecinos_nombres = gdf_vecinos['NOMGEO'].tolist() 
    
    # Almacenar en el diccionario
    vecinos_dict[row['NOMGEO']] = vecinos_nombres 

In [ ]:
def plot_estado_y_vecinos(estado, gdf, vecinos_dict):
    # Obtener la geometría del estado de interés
    estado_geom = gdf[gdf['NOMGEO'] == estado].geometry.iloc[0]
    
    # Obtener los vecinos del estado desde el diccionario
    vecinos = vecinos_dict.get(estado, [])
    
    # Filtrar los estados vecinos
    gdf_vecinos = gdf[gdf['NOMGEO'].isin(vecinos)]
    
    # Crear una figura y un eje para el gráfico
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Graficar el estado de interés con un color especial (por ejemplo, en azul)
    gdf[gdf['NOMGEO'] == estado].plot(ax=ax, color='lightblue', edgecolor='black', label=f'{estado}')
    
    # Graficar los estados vecinos con otro color (por ejemplo, en verde)
    gdf_vecinos.plot(ax=ax, color='lightgreen', edgecolor='black', label='Vecinos')
    
    # Personalizar el gráfico
    ax.set_title(f"Estado: {estado} y sus vecinos")
    
    # Mostrar el gráfico
    plt.show()

In [ ]:
#plot_estado_y_vecinos('México', gdf_mexico, vecinos_dict)
#plot_estado_y_vecinos('Guerrero', gdf_mexico, vecinos_dict)
#plot_estado_y_vecinos('Baja California', gdf_mexico, vecinos_dict)
#plot_estado_y_vecinos('Chihuahua', gdf_mexico, vecinos_dict)
#plot_estado_y_vecinos('Puebla', gdf_mexico, vecinos_dict)

## Segundo
### En cada grupo de estados vas a graficar las farmacias que se encuentran en el área, al igual que un círculo a su alrededor que tenga un radio de igual longitud que la distancia de esa farmacia a la siguiente farmacia más cercana.

In [ ]:
if gdf_mexico.crs != gdf_salud.crs:
    gdf_salud = gdf_salud.to_crs(gdf_mexico.crs)

In [ ]:
from shapely.ops import unary_union
import geopandas as gpd
import matplotlib.pyplot as plt

def plot_estado_y_vecinos_farmacias(estado, gdf, vecinos_dict):
    # Obtener la geometría del estado de interés
    estado_geom = gdf[gdf['NOMGEO'] == estado].geometry.iloc[0]
    
    # Obtener los vecinos del estado desde el diccionario
    vecinos = vecinos_dict.get(estado, [])
    
    # Filtrar los estados vecinos
    gdf_vecinos = gdf[gdf['NOMGEO'].isin(vecinos)]
    
    # Crear una figura y un eje para el gráfico
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Graficar el estado de interés con un color especial (por ejemplo, en azul)
    gdf[gdf['NOMGEO'] == estado].plot(ax=ax, color='lightblue', edgecolor='black', label=f'{estado}')
    
    # Graficar los estados vecinos con otro color (por ejemplo, en verde)
    gdf_vecinos.plot(ax=ax, color='lightgreen', edgecolor='black', label='Vecinos')
    
    
    # Filtrar las farmacias que están en el estado de interés
    gdf_farmacias_entidad = gdf_salud[gdf_salud['entidad'] == estado] 
    
    # Inicializar geometría para los vecinos
    geometrias_vecinos = list(gdf_farmacias_entidad.geometry)  # Comienza con las geometrías del estado
    
    # Añadir las geometrías de las farmacias en los vecinos
    for est in vecinos:
        geometrias_vecinos.extend(gdf_salud[gdf_salud['entidad'] == est].geometry)
    
    # Unir todas las geometrías
    geometria_unida_vecinos = unary_union(geometrias_vecinos)
    
    # Crear un GeoDataFrame con la geometría unida
    gdf_farmacias_vecinos = gpd.GeoDataFrame(geometry=[geometria_unida_vecinos])
    
    # Graficar las farmacias del estado sobre el mapa (en rojo)
    gdf_farmacias_entidad.plot(ax=ax, color='red', markersize=10, label='Farmacias')
    
    # Graficar las farmacias de los vecinos (en naranja)
    gdf_farmacias_vecinos.plot(ax=ax, color='orange', alpha=0.5, label='Farmacias Vecinas')
    
    # Personalizar el gráfico
    ax.set_title(f"{estado}, vecinos y farmacias")
    plt.show()

In [ ]:
#plot_estado_y_vecinos_farmacias('México', gdf_mexico, vecinos_dict)
#plot_estado_y_vecinos_farmacias('Guerrero', gdf_mexico, vecinos_dict)
#plot_estado_y_vecinos_farmacias('Baja California', gdf_mexico, vecinos_dict)
#plot_estado_y_vecinos_farmacias('Chihuahua', gdf_mexico, vecinos_dict)
#plot_estado_y_vecinos_farmacias('Puebla', gdf_mexico, vecinos_dict)

## Tercero
### Por grupo de estados debes calcular cual es la farmacia que en promedio está más alejada de el resto de farmacios.
### Debes graficar las distancias de esta farmacia al resto de las farmacias en el grupo de estados.

In [ ]:
def calcular_farmacia_mas_alejada_y_graficar(grupo_estados, gdf_salud):
    # Filtrar las farmacias dentro del grupo de estados 
    gdf_farmacias_grupo = gdf_salud[gdf_salud['entidad'].isin(grupo_estados)]
    # Si no hay farmacias, retornar
    if len(gdf_farmacias_grupo) == 0:
        print("No hay farmacias en este grupo de estados")
        return
        
    # Crear una matriz de distancias entre todas las farmacias
    n_farmacias = len(gdf_farmacias_grupo)
    distancias = np.zeros((n_farmacias, n_farmacias))
    
    # Calcular las distancias entre cada par de farmacias
    for i in range(n_farmacias):
        for j in range(i+1, n_farmacias):
            dist = gdf_farmacias_grupo.iloc[i].geometry.distance(gdf_farmacias_grupo.iloc[j].geometry)
            distancias[i,j] = dist
            distancias[j,i] = dist
    
    # Calcular la distancia promedio de cada farmacia al resto
    distancias_promedio = np.mean(distancias, axis=1)
    
    # Encontrar la farmacia con la mayor distancia promedio
    idx_mas_alejada = np.argmax(distancias_promedio)
    farmacia_mas_alejada = gdf_farmacias_grupo.iloc[idx_mas_alejada]
    
    # Obtener las distancias desde la farmacia más alejada a todas las demás
    distancias_a_mas_alejada = distancias[idx_mas_alejada]
    
    # Crear el gráfico
    plt.figure(figsize=(12, 6))
    
    # Histograma de distancias
    plt.hist(distancias_a_mas_alejada, bins=30, color='skyblue', alpha=0.7)
    plt.axvline(np.mean(distancias_a_mas_alejada), color='red', linestyle='dashed', 
                label=f'Distancia promedio: {np.mean(distancias_a_mas_alejada):.2f}')
    
    plt.title(f'Distribución de distancias desde la farmacia más alejada\nen {", ".join(grupo_estados)}')
    plt.xlabel('Distancia')
    plt.ylabel('Frecuencia')
    plt.legend()
    plt.show()
    
    # Crear mapa con la farmacia más alejada y las demás
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plotear todas las farmacias en azul
    gdf_farmacias_grupo.plot(ax=ax, color='blue', markersize=50, alpha=0.5, label='Farmacias')
    
    # Plotear la farmacia más alejada en rojo
    gpd.GeoDataFrame(geometry=[farmacia_mas_alejada.geometry]).plot(
        ax=ax, color='red', markersize=100, label='Farmacia más alejada')
    
    plt.title(f'Ubicación de la farmacia más alejada en {", ".join(grupo_estados)}')
    plt.legend()
    plt.show()
    
    print(f"La farmacia más alejada está en: {farmacia_mas_alejada['entidad']}")
    print(f"Con una distancia promedio de: {distancias_promedio[idx_mas_alejada]:.2f}")

In [ ]:
#grupo_estados = vecinos_dict['México']  # O el estado que desees analizar
#calcular_farmacia_mas_alejada_y_graficar(grupo_estados, gdf_salud)

#group_estados = vecinos_dict['Guerrero']
#calcular_farmacia_mas_alejada_y_graficar(group_estados, gdf_salud)

#group_estados = vecinos_dict['Baja California']
#calcular_farmacia_mas_alejada_y_graficar(group_estados, gdf_salud)

#group_estados = vecinos_dict['Chihuahua']
#calcular_farmacia_mas_alejada_y_graficar(group_estados, gdf_salud)

#group_estados = vecinos_dict['Puebla']
#calcular_farmacia_mas_alejada_y_graficar(group_estados, gdf_salud)


## Cuarto
### Calcula cual es el grupo de estados con mayor distancia promedio entre sus farmacias.
### Luego grafica el país completo con el shapefile de estados y grafica el grupo de estados con mayor distancia promedio entre farmacias de un color distinto.

In [ ]:
def calcular_distancia_promedio_farmacias_por_grupo(gdf_salud):
    # Diccionario para almacenar las distancias promedio por grupo
    distancias_promedio_grupos = {}
    
    # Iterar sobre cada grupo de estados
    for estado_central, grupo in vecinos_dict.items():
        # Filtrar las farmacias del grupo de estados
        gdf_farmacias_grupo = gdf_salud[gdf_salud['entidad'].isin(grupo)]
        
        # Si hay menos de 2 farmacias, no podemos calcular distancias
        if len(gdf_farmacias_grupo) < 2:
            continue
            
        # Calcular todas las distancias entre pares de farmacias
        distancias_totales = 0
        num_pares = 0
        
        for i, farmacia1 in gdf_farmacias_grupo.iterrows():
            for j, farmacia2 in gdf_farmacias_grupo.iterrows():
                if i < j:  # Evitar duplicados y comparación consigo misma
                    distancia = farmacia1.geometry.distance(farmacia2.geometry)
                    distancias_totales += distancia
                    num_pares += 1
        
        # Calcular el promedio
        if num_pares > 0:
            distancia_promedio = distancias_totales / num_pares
            distancias_promedio_grupos[estado_central] = distancia_promedio
    
    # Encontrar el grupo con la mayor distancia promedio
    estado_max_distancia = max(distancias_promedio_grupos.items(), key=lambda x: x[1])[0]
    grupo_max_distancia = vecinos_dict[estado_max_distancia]
    
    # Crear el mapa
    fig, ax = plt.subplots(figsize=(15, 10))
    
    # Plotear todos los estados en gris claro
    gdf_estados.plot(ax=ax, color='lightgray', edgecolor='black')
    
    # Plotear el grupo de estados con mayor distancia promedio en rojo
    gdf_estados[gdf_estados['NOMGEO'].isin(grupo_max_distancia)].plot(
        ax=ax, color='red', alpha=0.5, edgecolor='black')
    
    # Plotear las farmacias
    gdf_salud.plot(ax=ax, color='blue', markersize=10, alpha=0.5)
    
    plt.title(f'Grupo de estados con mayor distancia promedio entre farmacias\n'
              f'Estado central: {estado_max_distancia}, '
              f'Distancia promedio: {distancias_promedio_grupos[estado_max_distancia]:.2f}')
    
    # Agregar leyenda
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='red', alpha=0.5, label='Grupo con mayor distancia promedio'),
        Patch(facecolor='lightgray', label='Otros estados'),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', 
                  markersize=10, alpha=0.5, label='Farmacias')
    ]
    ax.legend(handles=legend_elements)
    
    plt.show()
    
    return estado_max_distancia, grupo_max_distancia, distancias_promedio_grupos[estado_max_distancia]


estado_max_distancia, grupo_max_distancia, distancia_promedio = calcular_distancia_promedio_farmacias_por_grupo(gdf_salud)
print(f"El grupo de estados con mayor distancia promedio entre farmacias es: {grupo_max_distancia}")
print(f"Con una distancia promedio de: {distancia_promedio:.2f}")

In [ ]:
#para subir mis archivos a github
#git add .
#git commit -m "comentario"
#git push origin main